In [64]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [94]:
from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

groq_api_key = os.getenv("GROQ_API_KEY")
#embeddings = HuggingFaceEmbeddings()

llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it",streaming=True)


In [95]:
from langchain.schema import AIMessage,HumanMessage,SystemMessage


In [96]:
speech = """ 
The conquered territory belongs to you; from your ranks the governors of it are chosen; already the greater part of its treasure passes into your hands, and when all Asia is overrun, then indeed I will go further than the mere satisfaction of our ambitions: 
the utmost hopes of riches or power which each one of you
"""

In [97]:
chat_message=[
    SystemMessage(content="You are expert with experise in summarizing speeched"),
    HumanMessage(content=f"Please provide a short and concisse summary of the follow speech:\n Text:{speech}")
]

In [69]:
llm.get_num_tokens(speech)

67

In [83]:
## getting the summary
llm(chat_message).content

'The speaker is addressing an audience of conquerors, promising them the spoils of war, including treasure and governance of conquered territories. They incite the audience by suggesting that once Asia is overrun, their ambitions will be satisfied with riches and power beyond their hopes.'

In [84]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

generictemplate="""
Write a summary of the following speech:
book:{speech}

"""

prompt=PromptTemplate(
    input_variables=['speech','language'],
    template=generictemplate
)
prompt

PromptTemplate(input_variables=['speech'], template='\nWrite a summary of the following speech:\nbook:{speech}\n\n')

In [72]:
complete_prompt=prompt.format(speech=speech,language="Bangla")
complete_prompt

'\nWrite a summary of the following speech:\nSpeech: \nThe conquered territory belongs to you; from your ranks the governors of it are chosen; already the greater part of its treasure passes into your hands, and when all Asia is overrun, then indeed I will go further than the mere satisfaction of our ambitions: \nthe utmost hopes of riches or power which each one of you\n\nTranslate the precise summary to Bangla\n'

In [73]:
llm.get_num_tokens(complete_prompt)

87

In [74]:
# llm_chain=LLMChain(llm=llm,prompt=prompt)
# summary=llm_chain.run({'speech':speech,'language':'Bangla'})
# summary

In [75]:
## StuffDocumentChain Text summarization

In [85]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("bdcon.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'source': 'bdcon.pdf', 'page': 0}, page_content="THE CONSTITUTION OF THE PEOPLE�S REPUBLIC OF\nBANGLADESH \n []\n   \n   \n    \n1[ BISMILLAH-AR-RAHMAN-AR-RAHIM\n \n    \n(In the name of Allah, the Beneficient, the Merciful)/\nIn the name of the Creator, the Merciful.]\nPREAMBLE\nWe, the people of Bangladesh, having proclaimed our independence on the 26th day\nof March, 1971 and through 2[ a historic struggle for national liberation], established\nthe independent, sovereign People's Republic of Bangladesh; \n3[ Pledging that the high ideals of nationalism, socialism, democracy and secularism,\nwhich inspired our heroic people to dedicate themselves to, and our brave martyrs to\nsacrifice their lives in, the national liberation struggle, shall be the fundamental\nprinciples of the Constitution;] \nFurther pledging that it shall be a fundamental aim of the State to realise through the\ndemocratic process a socialist society, free from exploitation a society in which t

In [86]:
template=""" Write a concise and short summary of the following book,
book :{text}


 """
prompt=PromptTemplate(input_variables=['text'],
                      template=template)

In [87]:
from langchain.chains.summarize import load_summarize_chain

In [88]:
chain=load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
output_summary=chain.run({'input_documents':docs})
output_summary



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise and short summary of the following book,
book :THE CONSTITUTION OF THE PEOPLE�S REPUBLIC OF
BANGLADESH 
 []
   
   
    
1[ BISMILLAH-AR-RAHMAN-AR-RAHIM
 
    
(In the name of Allah, the Beneficient, the Merciful)/
In the name of the Creator, the Merciful.]
PREAMBLE
We, the people of Bangladesh, having proclaimed our independence on the 26th day
of March, 1971 and through 2[ a historic struggle for national liberation], established
the independent, sovereign People's Republic of Bangladesh; 
3[ Pledging that the high ideals of nationalism, socialism, democracy and secularism,
which inspired our heroic people to dedicate themselves to, and our brave martyrs to
sacrifice their lives in, the national liberation struggle, shall be the fundamental
principles of the Constitution;] 
Further pledging that it shall be a fundamental aim of the State to realise through the
de

"The Constitution of the People's Republic of Bangladesh outlines the country's foundational principles, including nationalism, socialism, democracy, and secularism. It establishes Bangladesh as a unitary, independent, and sovereign republic with Islam as the state religion. The document also details the country's commitment to human rights, freedom, equality, and justice for all citizens. The constitution outlines the structure and functions of the republic, including the executive, legislative, and judicial branches. It also addresses local government, defense services, and the election commission. The document includes provisions for fundamental rights, such as equality before the law, freedom of movement, freedom of assembly, and freedom of thought and conscience. The constitution outlines the process for amending it and includes transitional and temporary provisions."

In [102]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("bdcon.pdf")
docs = loader.load_and_split()
docs
final_documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents
chunks_prompt="""
Please summarize the below text:
`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)
final_prompt='''
Provide the final summary of the entire book with these important points.
Add a information Title,Start the precise summary with an introduction and provide the summary in number 
points for the books.
book:{text}

'''
final_prompt_template=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_template
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below text:
`THE CONSTITUTION OF THE PEOPLE�S REPUBLIC OF
BANGLADESH 
 []
   
   
    
1[ BISMILLAH-AR-RAHMAN-AR-RAHIM
 
    
(In the name of Allah, the Beneficient, the Merciful)/
In the name of the Creator, the Merciful.]
PREAMBLE
We, the people of Bangladesh, having proclaimed our independence on the 26th day
of March, 1971 and through 2[ a historic struggle for national liberation], established
the independent, sovereign People's Republic of Bangladesh; 
3[ Pledging that the high ideals of nationalism, socialism, democracy and secularism,
which inspired our heroic people to dedicate themselves to, and our brave martyrs to
sacrifice their lives in, the national liberation struggle, shall be the fundamental
principles of the Constitution;] 
Further pledging that it shall be a fundamental aim of the State to realise through the
democratic process a social

"Please provide me with the text content of the amendments and changes to Article 150 that you want me to summarize. \n\nOnce you give me the text, I can:\n\n* Create a title that accurately reflects the content.\n* Write a concise introduction setting the context.\n* Summarize the key changes to Article 150 in numbered points, focusing on substitutions, insertions, omissions, and word changes.\n* Include the relevant Acts of Parliament and their dates for each amendment. \n\n\nI'm ready to help as soon as you provide the text! \n\n"

In [103]:
docs

[Document(metadata={'source': 'bdcon.pdf', 'page': 0}, page_content="THE CONSTITUTION OF THE PEOPLE�S REPUBLIC OF\nBANGLADESH \n []\n   \n   \n    \n1[ BISMILLAH-AR-RAHMAN-AR-RAHIM\n \n    \n(In the name of Allah, the Beneficient, the Merciful)/\nIn the name of the Creator, the Merciful.]\nPREAMBLE\nWe, the people of Bangladesh, having proclaimed our independence on the 26th day\nof March, 1971 and through 2[ a historic struggle for national liberation], established\nthe independent, sovereign People's Republic of Bangladesh; \n3[ Pledging that the high ideals of nationalism, socialism, democracy and secularism,\nwhich inspired our heroic people to dedicate themselves to, and our brave martyrs to\nsacrifice their lives in, the national liberation struggle, shall be the fundamental\nprinciples of the Constitution;] \nFurther pledging that it shall be a fundamental aim of the State to realise through the\ndemocratic process a socialist society, free from exploitation a society in which t

In [104]:
final_documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents

[Document(metadata={'source': 'bdcon.pdf', 'page': 0}, page_content="THE CONSTITUTION OF THE PEOPLE�S REPUBLIC OF\nBANGLADESH \n []\n   \n   \n    \n1[ BISMILLAH-AR-RAHMAN-AR-RAHIM\n \n    \n(In the name of Allah, the Beneficient, the Merciful)/\nIn the name of the Creator, the Merciful.]\nPREAMBLE\nWe, the people of Bangladesh, having proclaimed our independence on the 26th day\nof March, 1971 and through 2[ a historic struggle for national liberation], established\nthe independent, sovereign People's Republic of Bangladesh; \n3[ Pledging that the high ideals of nationalism, socialism, democracy and secularism,\nwhich inspired our heroic people to dedicate themselves to, and our brave martyrs to\nsacrifice their lives in, the national liberation struggle, shall be the fundamental\nprinciples of the Constitution;] \nFurther pledging that it shall be a fundamental aim of the State to realise through the\ndemocratic process a socialist society, free from exploitation a society in which t

In [105]:
len(final_documents)

96

In [106]:
chunks_prompt="""
Please summarize the below text:
text:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [107]:
final_prompt='''
Provide the final summary of the entire text with these important points.
Add a Motivation Title,Start the precise summary with an introduction and provide the summary in number 
points for the speech.
text:{text}

'''
final_prompt_template=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], template='\nProvide the final summary of the entire text with these important points.\nAdd a Motivation Title,Start the precise summary with an introduction and provide the summary in number \npoints for the speech.\ntext:{text}\n\n')

In [108]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below text:
text:`THE CONSTITUTION OF THE PEOPLE�S REPUBLIC OF
BANGLADESH 
 []
   
   
    
1[ BISMILLAH-AR-RAHMAN-AR-RAHIM
 
    
(In the name of Allah, the Beneficient, the Merciful)/
In the name of the Creator, the Merciful.]
PREAMBLE
We, the people of Bangladesh, having proclaimed our independence on the 26th day
of March, 1971 and through 2[ a historic struggle for national liberation], established
the independent, sovereign People's Republic of Bangladesh; 
3[ Pledging that the high ideals of nationalism, socialism, democracy and secularism,
which inspired our heroic people to dedicate themselves to, and our brave martyrs to
sacrifice their lives in, the national liberation struggle, shall be the fundamental
principles of the Constitution;] 
Further pledging that it shall be a fundamental aim of the State to realise through the
democratic process a s

"## Bangladesh: A Nation Forged in Law\n\n**Motivation:**  This summary explores the foundational legal framework of Bangladesh, highlighting its structure, key institutions, and the mechanisms for safeguarding rights and governance.\n\n**Introduction:**  This document outlines the core principles and operational aspects of Bangladesh's legal system, emphasizing the balance of power, judicial independence, and the role of the Constitution in shaping the nation's governance.  \n\n**Key Points:**\n\n1. **Constitutional Foundation:** The Constitution of Bangladesh is the supreme law, establishing the framework for government, defining fundamental rights, and outlining the separation of powers.  It is a living document, amendable to reflect the nation's evolving needs.\n\n2. **Judicial Independence:** Bangladesh's judicial system is designed to be independent and impartial. The Supreme Court, comprising the High Court Division and Appellate Division, ensures the interpretation and applicat

In [109]:
output

"## Bangladesh: A Nation Forged in Law\n\n**Motivation:**  This summary explores the foundational legal framework of Bangladesh, highlighting its structure, key institutions, and the mechanisms for safeguarding rights and governance.\n\n**Introduction:**  This document outlines the core principles and operational aspects of Bangladesh's legal system, emphasizing the balance of power, judicial independence, and the role of the Constitution in shaping the nation's governance.  \n\n**Key Points:**\n\n1. **Constitutional Foundation:** The Constitution of Bangladesh is the supreme law, establishing the framework for government, defining fundamental rights, and outlining the separation of powers.  It is a living document, amendable to reflect the nation's evolving needs.\n\n2. **Judicial Independence:** Bangladesh's judicial system is designed to be independent and impartial. The Supreme Court, comprising the High Court Division and Appellate Division, ensures the interpretation and applicat

In [110]:
len(output)

2207

In [ ]:
# To create a system where you use agents to retrieve the best answers from a large file using FAISS-based embeddings, you're essentially combining the principles of Retrieval-Augmented Generation (RAG) with the flexibility of agents. Here's how you can approach this:

# 1. Initial Setup
# Text Embedding and Splitting: You've already embedded your text data and split it into manageable chunks, storing these embeddings in FAISS for efficient retrieval.
# Traditional RAG Setup: In a traditional RAG setup, you'd use a retrieval chain that queries the FAISS index to get relevant document chunks, and then a generation model (like GPT) to generate answers based on those retrieved documents.
# 2. Introducing Agents
# Agent Concept: Agents in this context are intelligent components that can perform specific tasks, like retrieving information, generating text, or even orchestrating a series of steps based on user queries.
# Tooling: For building agents, you can use frameworks like LangChain, which allow you to create agents that interact with your FAISS-based retrieval system.
# 3. Creating the Agent Workflow
# Agent Design:

# User Query Handling: The agent takes the user’s question and processes it.
# Retrieval Task: The agent sends the query to the retrieval system, which uses FAISS to find the most relevant text chunks.
# Generation Task: The agent then passes these retrieved chunks to a language model to generate a coherent and contextually accurate response.
# Final Response: The agent combines the model's output and returns the best possible answer to the user.
# Example Flow:

# Query Handling: Agent receives the query "What is the impact of climate change on agriculture?"
# FAISS Retrieval: Agent queries the FAISS index, retrieving relevant text chunks.
# Answer Generation: Agent forwards the retrieved text to a language model (e.g., GPT-3/4), which generates an answer.
# Return Response: Agent returns the generated answer to the user.
# 4. Implementing the Agent
# Using LangChain:

# Create a custom agent that can handle multiple tasks, including retrieval and generation.
# Define tools within LangChain for retrieval and text generation. These tools can include FAISS retrieval and the language model.
# Orchestrate tasks: The agent will orchestrate the process, ensuring that the query is handled efficiently and the best answer is generated.
# Agent Execution: The agent is triggered when a user asks a question. It dynamically decides the best course of action—retrieving information, generating text, or even performing additional tasks like summarizing or ranking responses.

# 5. Advantages Over Traditional RAG:
# Flexibility: Agents allow for more complex workflows, such as multi-step retrieval, post-processing of answers, or integrating additional tools (e.g., web scraping or databases).
# Customizability: You can fine-t

# You said:
# give me the code
# ChatGPT said:
# ChatGPT
# Certainly! Below is a basic code structure to create an agent-based system that uses FAISS for embedding retrieval and then leverages LangChain's ConversationalRetrievalChain to handle questions. This setup will allow you to use an agent that queries the FAISS index and generates answers based on the retrieved context.

# 1. Setup FAISS Index
# First, ensure you have your text data embedded and stored in FAISS.

# python
# Copy code
# from langchain.vectorstores import FAISS
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.document_loaders import TextLoader

# # Load your large text file
# loader = TextLoader('large_text_file.txt')
# documents = loader.load()

# # Split the text into manageable chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# texts = text_splitter.split_documents(documents)

# # Initialize OpenAI embeddings
# embeddings = OpenAIEmbeddings()

# # Create a FAISS index
# vectorstore = FAISS.from_documents(texts, embeddings)

# # Save the FAISS index
# vectorstore.save_local("faiss_index")
# 2. Create an Agent for RAG
# Now, let's create an agent that can use the FAISS index to retrieve relevant chunks and generate an answer.

# python
# Copy code
# from langchain.chains import ConversationalRetrievalChain
# from langchain.llms import OpenAI
# from langchain.agents import initialize_agent, Tool

# # Load the FAISS index
# vectorstore = FAISS.load_local("faiss_index", embeddings)

# # Create a Conversational Retrieval Chain
# qa_chain = ConversationalRetrievalChain.from_llm(
#     llm=OpenAI(),
#     retriever=vectorstore.as_retriever(),
# )

# # Define a Tool for the agent to use the QA chain
# tools = [
#     Tool(
#         name="FAISS_QA_Tool",
#         func=qa_chain.run,
#         description="Use this tool to answer questions based on the text data."
#     )
# ]

# # Initialize the agent with the tools
# agent = initialize_agent(tools, OpenAI(), agent_type="zero-shot-react-description", verbose=True)

# # Start asking questions
# response = agent.run("What is the key point in the document?")
# print(response)
# 3. Running the Agent
# You can now use the agent.run() method to ask questions, and the agent will retrieve the most relevant chunks from the FAISS index and generate an answer based on them.

# 4. Explanation
# Text Loader & Splitter: Loads and splits the text into chunks for embedding.
# FAISS: Manages the embeddings for efficient retrieval.
# ConversationalRetrievalChain: Combines retrieval with LLM-based generation.
# Agent: An agent wraps around the retrieval chain and allows you to interact with it in a more dynamic way.
# This setup allows for a more flexible and dynamic interaction with your data, where the agent can leverage the FAISS index to generate the best possible answers based on the context retrieved.






